In [1]:
!pip install flair

  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.7 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
import flair.datasets
from flair.data import Corpus
corpus = flair.datasets.NER_CHINESE_WEIBO()
print(corpus)

2022-06-17 05:04:55,149 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.train not found in cache, downloading to /tmp/tmpx78152y1


462545B [00:00, 43442481.61B/s]          

2022-06-17 05:04:55,187 copying /tmp/tmpx78152y1 to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.train
2022-06-17 05:04:55,192 removing temp file /tmp/tmpx78152y1


2022-06-17 05:04:55,410 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.test not found in cache, downloading to /tmp/tmpz36nv7rm


94016B [00:00, 28850723.21B/s]          

2022-06-17 05:04:55,442 copying /tmp/tmpz36nv7rm to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.test
2022-06-17 05:04:55,445 removing temp file /tmp/tmpz36nv7rm


2022-06-17 05:04:55,657 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.dev not found in cache, downloading to /tmp/tmpnagdn2zr


91510B [00:00, 28164129.66B/s]          

2022-06-17 05:04:55,689 copying /tmp/tmpnagdn2zr to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.dev
2022-06-17 05:04:55,692 removing temp file /tmp/tmpnagdn2zr
2022-06-17 05:04:55,694 Reading data from /root/.flair/datasets/ner_chinese_weibo
2022-06-17 05:04:55,695 Train: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.train
2022-06-17 05:04:55,696 Dev: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.dev
2022-06-17 05:04:55,697 Test: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.test


Corpus: 1350 train + 270 dev + 270 test sentences


In [ ]:
import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings, BytePairEmbeddings

tag_type = 'ner'
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

# For an even faster training time, you can comment out the BytePairEmbeddings
# Note: there will be a small drop in performance if you do so.
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('zh-crawl'),
    BytePairEmbeddings('zh'),
    BertEmbeddings('bert-base-chinese'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('/content/model/',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='cpu')

In [39]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentences = [
  '我 和 华 莱 士 在 美 国 谈 笑 风 生 !',
  '狼 王 加 内 特 在 兴 安 岭 看 狼 群 。',
  '上 海 自 来 水 来 自 海 上 。',
  '马 云 说 阿 里 的 996 工 作 制 是 福 报',
  '马 云 和 阿 里 打 拳 击',
  '我 带 小 黄 在 上 海 人 民 广 场 的 百 联 商 店 一 起 吃 炸 鸡'
]
tagger: SequenceTagger = SequenceTagger.load("/content/model/final-model.pt")
for input_sentence in input_sentences:
  sentence: Sentence = Sentence(input_sentence)
  tagger.predict(sentence)
  print(sentence.to_tagged_string())

2022-06-17 06:20:28,870 loading file /content/model/final-model.pt
2022-06-17 06:20:31,691 SequenceTagger predicts: Dictionary with 35 tags: O, S-PER.NOM, B-PER.NOM, E-PER.NOM, I-PER.NOM, S-PER.NAM, B-PER.NAM, E-PER.NAM, I-PER.NAM, S-GPE.NAM, B-GPE.NAM, E-GPE.NAM, I-GPE.NAM, S-ORG.NAM, B-ORG.NAM, E-ORG.NAM, I-ORG.NAM, S-LOC.NAM, B-LOC.NAM, E-LOC.NAM, I-LOC.NAM, S-LOC.NOM, B-LOC.NOM, E-LOC.NOM, I-LOC.NOM, S-ORG.NOM, B-ORG.NOM, E-ORG.NOM, I-ORG.NOM, S-GPE.NOM, B-GPE.NOM, E-GPE.NOM, I-GPE.NOM, <START>, <STOP>
Sentence: "我 和 华 莱 士 在 美 国 谈 笑 风 生 !" → ["华 莱 士"/PER.NAM, "美 国"/GPE.NAM]
Sentence: "狼 王 加 内 特 在 兴 安 岭 看 狼 群 。" → ["加 内 特"/PER.NAM, "兴 安 岭"/LOC.NAM]
Sentence: "上 海 自 来 水 来 自 海 上 。" → ["上 海"/GPE.NAM]
Sentence: "马 云 说 阿 里 的 996 工 作 制 是 福 报" → ["马 云"/PER.NAM, "阿 里"/ORG.NAM]
Sentence: "马 云 和 阿 里 打 拳 击" → ["马 云"/PER.NAM, "阿 里"/PER.NAM]
Sentence: "我 带 小 黄 在 上 海 人 民 广 场 的 百 联 商 店 一 起 吃 炸 鸡" → ["小 黄"/PER.NAM, "上 海"/GPE.NAM, "人 民 广 场"/LOC.NAM, "百 联 商 店"/ORG.NAM]
